In [1]:
import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t_spin, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import generate_Hamiltonian_with_occupation_number, generate_reduced_H_non_ortho
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn


Load and initialize datas: 

no is number of spatial orbitals

L is number of spinor orbitals, L = 2*no

t_spin is one-body integral in Chemist's notation (considering spins)

g_spin is two-body integral in Chemist's notation (considering spins)

X_mo and Z_mo are THC tensors, X_mo_up/down are X_mo considering spins

r_THC is THC rank

In [2]:
#load integrals
with h5py.File("data_water/eri_water.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("data_water/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

7
(7, 28)
(28, 28)
rl errV: 2.8386751875274264e-12
abs errV: 2.0615721155266396e-11
errt: 7.097049412242525e-13
errh: 2.585427402664151e-13
errht: 9.079449636842276e-14


These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [3]:
#load Hamiltonian generated by above coefficients
H_correct = scipy.io.mmread('data_water/H_water_correct.mtx').tocsr()
e1, v1 = sparse.linalg.eigsh(H_correct, which = 'SA', k = 1)
e_ground = e1

H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)

/home/ge49cag/.local/lib/python3.10/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


We can calculate elements in reduced Hamiltonian using conventional MPO.

Since we only need to store ONE block during contraction, memory needed is only $\mathcal{O}(L^2 M^2)$.

Create conventional mpo for molecular Hamiltonian:

In [4]:
h1_spin = get_h1_spin(h1)
g_spin = get_g_spin(eri)
g_spin_phy =  g_spin.transpose(0, 2, 1, 3)
mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
print(mpo_ref.bond_dims)

[1, 4, 16, 39, 58, 75, 96, 115, 96, 75, 58, 39, 16, 4, 1]


Generate THC-MPO by layers, using THC tensors. 
t_spin is used to create MPO for kinetic term.
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.

In [5]:
#generate thc_mpo
t_spin = get_t_spin(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t_spin)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

For 1st excited state, please use single-excited Hatree-Fock state as initial state, or even superposition of several single-excited Hatree-Fock states as initial state.

In [6]:
hartree_state = generate_single_state(14, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])
hartree_state_vector = hartree_state.as_vector()

We generate a group of orthogonal Krylov vectors using THC-MPO, with bond dim 40 for Krylov vectors. The vectors are stored in the folder = 'foldername', thus you don't have to generate them again for next time use. 

In [7]:
N_Krylov = 80
psi_original = copy.deepcopy(hartree_state)
max_bond_Krylov = 80
#max_bond_Krylov = 75
trunc_tol = 0
foldername = f"water_Krylov"
#Krylov vectors are included in data, you dont have to run generate it. ofc, you can -regenerate it to verify the algorithm using the following code:
#generate_krylov_space_in_disk(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_thc, foldername)

#it indicates that even though during the calculation the bond dims exceed 40, but we only need 37 for Krylov vectors.

Make use of method proposed in https://journals.aps.org/prb/abstract/10.1103/PhysRevB.85.205119 to improve the orthogonality of Krylov vectors. 

In [8]:
N_use = 50
# C = coeff_canonical_orthogonalization(N_use, foldername)
# vector_list = generate_re_ortho_space_with_coeff(N_use, C, foldername)
#vector_list = generate_re_ortho_space(N_use, foldername)
H_reduced_non_rotho = generate_reduced_H_non_ortho(N_use, foldername, mpo_ref)
W = get_W(N_use, foldername)
coeff = get_S(W)
coeff = np.array(coeff)
H_reduced = np.einsum('ik, kl, jl -> ij', coeff.conj(), H_reduced_non_rotho, coeff)
H_reduced = remain_only_tridiagonal_elements(H_reduced)

/work_fast/ge49cag/pytenet_thc/pytenet/global_krylov_method.py:193: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc/pytenet/global_krylov_method.py:106: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = np.vdot(temp1.as_vector(), temp2.as_vector())


After achieving Krylov vectors, we implement Lanczos algorithm using these Krylov vectors. In this notebook, all the expectation values are calculated using ED for convenience. Since calculating the expectation value (contracting the tensor network) doesn't bring new errors, it makes no difference whether we use tensor network or ED to calculate the matrix elements for reduced Hamiltonian.

In [9]:
N_list = [0]
mps_start = copy.deepcopy(hartree_state)
error_list = [np.vdot(mps_start.as_vector(), H_correct_10e@(mps_start.as_vector())) - e_ground]

In [12]:
for N in range(5, N_use+1, 5):
    N_list.append(N)
    H_part = H_reduced[:N, :N]
    e_rotate, v_rotate = np.linalg.eigh(H_part)

    temp = v_rotate[:,0]
    temp = coeff[:N, :N].transpose(1,0)@temp
    temp = H_reduced_non_rotho[:N, :N]@temp
    temp = coeff[:N, :N].conj()@temp
    e_rotate_ground = ((v_rotate[:,0].reshape(1, N)).conj())@temp
    #e_rotate_ground = np.einsum(', ,  -> ', v_rotate_ground_coeff.conj(), H_reduced_non_rotho, v_rotate_ground_coeff)
    #e_new = np.vdot(v_rotate_ground, H_correct_10e@v_rotate_ground)
    error_list.append(e_rotate_ground - e_ground)
    
    print(e_rotate_ground - e_ground)
    #result gets worse when N_krylov increase, since the orthogonality gets worse.
    #One need to re-start the Lanczos to solve this issue.

[0.35244435]
[0.02484897]
[0.00195854]
[7.230799e-05]
[8.5151126e-07]
[1.42242413e-08]
[1.02488684e-10]
[-1.13686838e-13]
[-7.10542736e-14]
[-8.52651283e-14]


In [11]:
#to do: calculate H10 case.
#we can do NH3 quickly.